### Bronze Layer: Ingest lap_times

Load raw CSV files from landing zone folder and save as parquet in bronze/raw layer.

In [1]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

/home/ricardo/Documentos/Projects/dab-ci-cd/dab_project/.venv/lib/python3.12/site-packages/databricks/sdk/_widgets/__init__.py:71: UserWarning: 
To use databricks widgets interactively in your notebook, please install databricks sdk using:
	pip install 'databricks-sdk[notebook]'
Falling back to default_value_only implementation for databricks widgets.
  warnings.warn(


In [2]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import current_timestamp, lit

In [3]:
from formula1.formula1_constants import landing_folder_path, raw_folder_path

##### Step 1 - Define schema and read CSV files from landing zone folder

In [4]:
lap_times_schema = StructType(fields=[
    StructField("raceId", IntegerType(), False),
    StructField("driverId", IntegerType(), True),
    StructField("lap", IntegerType(), True),
    StructField("position", IntegerType(), True),
    StructField("time", StringType(), True),
    StructField("milliseconds", IntegerType(), True)
])

In [5]:
# Read all CSV files from the lap_times folder
lap_times_df = spark.read \
    .schema(lap_times_schema) \
    .csv(f"{landing_folder_path}/lap_times/")

##### Step 2 - Add ingestion metadata

In [6]:
lap_times_df = lap_times_df \
    .withColumn("ingestion_date", current_timestamp()) \
    .withColumn("data_source", lit(v_data_source))

##### Step 3 - Write to bronze/raw layer as parquet

In [7]:
lap_times_df.write.mode("overwrite").parquet(f"{raw_folder_path}/lap_times")